In [1]:
import test_config as cfg
import numpy as np
import itertools, functools

%matplotlib inline
import matplotlib.pyplot as plt
import os, os.path
import pickle
import io

import json

import skimage, skimage.io, skimage.transform, skimage.filters
import sklearn, sklearn.metrics

import importlib
import time
import sys
sys.path.append('../src/')
import modutils
import word_processing as wp
import tqdm

In [17]:
importlib.reload(wp)

<module 'word_processing' from '../src\\word_processing.py'>

In [2]:
%%time
fname = 'D:/Data/bujo_sample_v2/dataset.json'
extraction_path = os.path.join(os.path.dirname(fname),
                               os.path.basename(fname).split('.')[0])
with open(fname, 'r', encoding='utf-8') as f:
    src = json.load(f)
    
words = wp.extract_words_from_dataset(src, (1,))

Wall time: 132 ms


In [3]:
len(words)

3630

In [8]:
fin_words = [(w[0].lower(), w[1]) for w in words]

In [18]:
transforms_rotation = wp.generate_rotations(-5, 5, 5)
transforms_cutoff = wp.generate_cutoffs(0.6, 0.8, 3)
transforms_scales = wp.generate_scales(0.85, 1.15, 3, fill=0.0)
combinations_all = wp.generate_combinations(transforms_rotation, transforms_cutoff, transforms_scales)
fixed_post = [{'type':'trimx'}, {'type':'trimy'}, {'type':'resize', 'y':32}, {'type':'invert'}]
transforms_all = wp.append_transformations(combinations_all, post_transforms=fixed_post)

In [11]:
train_ids = np.random.choice(np.arange(len(fin_words)), size=int(len(fin_words)*0.6), replace=False)
train_charset = functools.reduce(set.union, [set(fin_words[i][0]) for i in train_ids])
valid_ids = [i for i in range(len(fin_words)) if i not in train_ids]
train_set = [fin_words[i] for i in train_ids]
valid_set = [fin_words[i] for i in valid_ids if set.issubset(set(fin_words[i][0]), train_charset)]

In [12]:
len(train_set), len(valid_set)

(2178, 1450)

In [13]:
def save_set(path, words, transforms, dst_fname):
    dst_path, dst_name_ext = os.path.split(dst_fname)
    if '.' not in dst_name_ext:
        raise Exception('Not specified extension of file!')
    dst_name = '.'.join(dst_name_ext.split('.')[:-1])
    os.makedirs(os.path.join(dst_path, dst_name), exist_ok=True)
    with open(dst_fname, 'w') as f:
        for (i, (word, fname)) in enumerate(tqdm.tqdm(words)):
            src_image = skimage.io.imread(os.path.join(path, fname), as_grey=True)
            for (j, transform) in enumerate(transforms):
                res_image = wp.perform_transform(src_image, transform)
                loc_fname = 'w{:06d}t{:03d}'.format(i, j)
                res_fname = '{}/{}.png'.format(dst_name, loc_fname)
                f.write('{} {}\n'.format(loc_fname, word))
                skimage.io.imsave(os.path.join(dst_path, res_fname), res_image)

In [ ]:
%%time
save_set(extraction_path, train_set, transforms_all, 'D:/Data/HTR-attrs/train.txt')



  0%|                                                                                         | 0/2178 [00:00<?, ?it/s]

  0%|                                                                                 | 1/2178 [00:01<46:54,  1.29s/it]

  0%|                                                                                 | 2/2178 [00:02<41:08,  1.13s/it]

  0%|                                                                                 | 3/2178 [00:03<42:57,  1.19s/it]

  0%|▏                                                                                | 4/2178 [00:05<54:23,  1.50s/it]

  0%|▏                                                                                | 5/2178 [00:07<56:57,  1.57s/it]

  0%|▏                                                                                | 6/2178 [00:08<57:11,  1.58s/it]

  0%|▎                                                                                | 7/2178 [00:09<51:25,  1.42s/it]

  0%|▎                        